Submitted by Libin Kutty - 226051


Team Members

*   Libin Kutty - Group 2
*   Ritu Gahir - Group 3
*   Viju Sudhi - Group 10


In [0]:
%tensorflow_version 2.x

In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.utils import to_categorical

In [0]:
@tf.function
def load_data():
  (train_imgs, train_lbls), (test_imgs, test_lbls) = tf.keras.datasets.cifar10.load_data()
  train_imgs = (train_imgs.astype(np.float32) / 255.).reshape((-1, 32, 32, 3))
  train_lbls = train_lbls.astype(np.int32)


  test_imgs = (test_imgs.astype(np.float32) / 255.).reshape((-1, 32, 32, 3))
  test_lbls = test_lbls.astype(np.int32)

  batch_size = 128

  train_data = tf.data.Dataset.from_tensor_slices((train_imgs, train_lbls))

  train_data = train_data.shuffle(train_imgs.shape[0])
  train_data = train_data.batch(batch_size)
  train_data = train_data.repeat(5)


  test_data = tf.data.Dataset.from_tensor_slices((test_imgs, test_lbls))
  test_data = test_data.batch(batch_size)

  return train_data, test_data

## **Without tf.function**

In [0]:
class CNNModel(object):
  def __init__(self):

    self.loss_fn = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
    self.train_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
    self.test_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
    self.opt = tf.optimizers.Adam()

  def build(self):
    self.model = Sequential()

    self.model.add(Conv2D(32, (3, 3), input_shape=(32,32,3))) #train_imgs.shape[1:]
    self.model.add(Activation('relu'))
    self.model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.2))

    self.model.add(Conv2D(64, (3, 3)))
    self.model.add(Activation('relu'))
    self.model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.2))

    self.model.add(Conv2D(128, (3, 3)))
    self.model.add(Activation('relu'))
    self.model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.2))

    self.model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

    #model.add(Dense(64))
    #model.add(Dropout(0.2))

    self.model.add(Dense(128))
    #model.add(Dropout(0.2))

    self.model.add(Dense(10))
    self.model.add(Activation('sigmoid'))

    return self.model

  def train(self, train_data):

    for step, (img_batch, lbl_batch) in enumerate(train_data):

        with tf.GradientTape() as tape:
            logits = self.model(img_batch)
            xent = self.loss_fn(lbl_batch, logits)

        varis = self.model.trainable_variables
        grads = tape.gradient(xent, varis)

        self.opt.apply_gradients(zip(grads, varis))
        
        self.train_acc_metric(lbl_batch, logits)
        
        if tf.equal(step % 500, 0): #not step % 500:
          acc = self.train_acc_metric.result()
          tf.print("Loss: {} Accuracy: {}".format(xent, acc))
          self.train_acc_metric.reset_states()

  def test(self,test_data):
    for img_batch, lbl_batch in test_data:
        self.test_acc_metric(lbl_batch, self.model(img_batch))
    tf.print("Test acc: {}".format(self.test_acc_metric.result()))

In [0]:
train_data, test_data = load_data()

cnnmodel = CNNModel()

cnnmodel.build()

start = time.time()
cnnmodel.train(train_data)
stop = time.time()

print("took {} seconds\n".format(stop-start))

170500096/170498071 [==============================] - 4s 0us/step
Loss: 2.3069443702697754 Accuracy: 0.09375
Loss: 1.89448881149292 Accuracy: 0.35925382375717163
Loss: 1.8808059692382812 Accuracy: 0.4953089952468872
Loss: 1.8090543746948242 Accuracy: 0.5432824492454529
took 452.248526096344 seconds



# **With tf.function on train method**

In [0]:
class CNNModel(object):
  def __init__(self):

    self.loss_fn = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
    self.train_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
    self.test_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
    self.opt = tf.optimizers.Adam()

  def build(self):
    self.model = Sequential()

    self.model.add(Conv2D(32, (3, 3), input_shape=(32,32,3))) #train_imgs.shape[1:]
    self.model.add(Activation('relu'))
    self.model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.2))

    self.model.add(Conv2D(64, (3, 3)))
    self.model.add(Activation('relu'))
    self.model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.2))

    self.model.add(Conv2D(128, (3, 3)))
    self.model.add(Activation('relu'))
    self.model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.2))

    self.model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

    #model.add(Dense(64))
    #model.add(Dropout(0.2))

    self.model.add(Dense(128))
    #model.add(Dropout(0.2))

    self.model.add(Dense(10))
    self.model.add(Activation('sigmoid'))

    return self.model

  @tf.function
  def train(self, train_data):

    for step, (img_batch, lbl_batch) in enumerate(train_data):

        with tf.GradientTape() as tape:
            logits = self.model(img_batch)
            xent = self.loss_fn(lbl_batch, logits)

        varis = self.model.trainable_variables
        grads = tape.gradient(xent, varis)

        self.opt.apply_gradients(zip(grads, varis))
        
        self.train_acc_metric(lbl_batch, logits)
        
        if tf.equal(step % 500, 0): #not step % 500:
          acc = self.train_acc_metric.result()
          tf.print('Loss:', xent)
          tf.print('Accuracy:', acc)
#          tf.print("Loss: {} Accuracy: {}".format(xent, acc))
          self.train_acc_metric.reset_states()

  @tf.function
  def test(self,test_data):
    for img_batch, lbl_batch in test_data:
        self.test_acc_metric(lbl_batch, self.model(img_batch))
    tf.print('Test Accuracy:', self.test_acc_metric.result() )
    #tf.print("Test acc: {}".format(self.test_acc_metric.result()))

In [0]:
train_data, test_data = load_data()

cnnmodel = CNNModel()

cnnmodel.build()

start = time.time()
cnnmodel.train(train_data)
stop = time.time()

print("took {} seconds\n".format(stop-start))

Loss: 2.30225658
Accuracy: 0.1015625
Loss: 1.94462824
Accuracy: 0.348089188
Loss: 1.92534614
Accuracy: 0.471681893
Loss: 1.76662922
Accuracy: 0.515980721
took 314.63280510902405 seconds



**Time**


*   without tf.function - 450 seconds
*   with tf.function - 315 seconds


In [0]:
cnnmodel.test(test_data)

Test Accuracy: 0.5416


In [0]:
  (train_imgs, train_lbls), (test_imgs, test_lbls) = tf.keras.datasets.cifar10.load_data()
  train_imgs = (train_imgs.astype(np.float32) / 255.).reshape((-1, 32, 32, 3))
  train_lbls = train_lbls.astype(np.int32)


  test_imgs = (test_imgs.astype(np.float32) / 255.).reshape((-1, 32, 32, 3))
  test_lbls = test_lbls.astype(np.int32)

  batch_size = 128

  train_data = tf.data.Dataset.from_tensor_slices((train_imgs, train_lbls))

  train_data = train_data.shuffle(train_imgs.shape[0])
  train_data = train_data.batch(batch_size)
  train_data = train_data.repeat(3)


  test_data = tf.data.Dataset.from_tensor_slices((test_imgs, test_lbls))
  test_data = test_data.batch(batch_size)

# **DenseNet**

In [0]:
train_steps = 500

inp = tf.keras.layers.Input((32, 32, 3))

initial_conv = tf.keras.layers.Conv2D(32, 3, activation=tf.nn.relu, padding="same")(inp)

#Dense Block 1
conv1_1 = tf.keras.layers.Conv2D(32, 3, activation=tf.nn.relu, padding="same")(initial_conv)
conv1_2 = tf.keras.layers.Conv2D(32, 3, activation=tf.nn.relu, padding="same")(conv1_1)
conc1 = tf.keras.layers.concatenate([conv1_1, conv1_2])
conv1_3 = tf.keras.layers.Conv2D(32, 3, activation=tf.nn.relu, padding="same")(conc1)
conc2 = tf.keras.layers.concatenate([conv1_1, conv1_2, conv1_3])
conv1_4 = tf.keras.layers.Conv2D(32, 3, activation=tf.nn.relu, padding="same")(conc2)

#Transition Layer 1
t_conv1 = tf.keras.layers.Conv2D(32, 3, activation=tf.nn.relu, padding="same")(conv1_4)
pool1 = tf.keras.layers.MaxPool2D(3, 1, padding="same")(t_conv1)

#Dense Block 2
conv2_1 = tf.keras.layers.Conv2D(64, 3, activation=tf.nn.relu, padding="same")(pool1)
conv2_2 = tf.keras.layers.Conv2D(64, 3, activation=tf.nn.relu, padding="same")(conv2_1)
conc2_1 = tf.keras.layers.concatenate([conv2_1, conv2_2])
conv2_3 = tf.keras.layers.Conv2D(64, 3, activation=tf.nn.relu, padding="same")(conc2_1)
conc2_2 = tf.keras.layers.concatenate([conv2_1, conv2_2, conv2_3])
conv2_4 = tf.keras.layers.Conv2D(64, 3, activation=tf.nn.relu, padding="same")(conc2_2)

#Transition Layer 2
t_conv2 = tf.keras.layers.Conv2D(64, 3, activation=tf.nn.relu, padding="same")(conv2_4)
pool2 = tf.keras.layers.MaxPool2D(3, 1, padding="same")(t_conv2)

#Dense Block 3
#conv3_1 = tf.keras.layers.Conv2D(32, 3, activation=tf.nn.relu, padding="same")(pool2)
#conv3_2 = tf.keras.layers.Conv2D(32, 3, activation=tf.nn.relu, padding="same")(conv3_1)
#conc3_1 = tf.keras.layers.concatenate([conv3_1, conv3_2])
#conv3_3 = tf.keras.layers.Conv2D(32, 3, activation=tf.nn.relu, padding="same")(conc3_1)
#conc3_2 = tf.keras.layers.concatenate([conv3_1, conv3_2, conv3_3])
#conv3_4 = tf.keras.layers.Conv2D(32, 3, activation=tf.nn.relu, padding="same")(conc3_2)

#Transition Layer 3
#t_conv3 = tf.keras.layers.Conv2D(32, 3, activation=tf.nn.relu, padding="same")(conv3_4)
#pool3 = tf.keras.layers.MaxPool2D(3, 1, padding="same")(t_conv3)

flat = tf.keras.layers.Flatten()(pool2)
out = tf.keras.layers.Dense(10)(flat)

model = tf.keras.Model(inp, out, name='DenseNet')

model.summary()

opt = tf.optimizers.Adam()

loss_fn = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

train_acc_metric = tf.metrics.SparseCategoricalAccuracy()

Model: "DenseNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_25 (Conv2D)              (None, 32, 32, 32)   896         input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_26 (Conv2D)              (None, 32, 32, 32)   9248        conv2d_25[0][0]                  
__________________________________________________________________________________________________
conv2d_27 (Conv2D)              (None, 32, 32, 32)   9248        conv2d_26[0][0]                  
___________________________________________________________________________________________

In [0]:
@tf.function
def train_step(imgs, lbls):
    with tf.GradientTape() as tape:
        logits = model(imgs)
        xent = loss_fn(lbls, logits)

    varis = model.trainable_variables
    grads = tape.gradient(xent, varis)
    opt.apply_gradients(zip(grads, varis))

    return xent, logits

In [0]:
import time
start = time.time()

for step, (img_batch, lbl_batch) in enumerate(train_data):

    xent, logits = train_step(img_batch, lbl_batch)

    if not step % 100:
        train_acc_metric(lbl_batch, logits)
        acc = train_acc_metric.result()
        print("Loss: {} Accuracy: {}".format(xent, acc))
        train_acc_metric.reset_states()

        stop = time.time()
        print("took {} seconds\n".format(stop-start))
        start = time.time()

Loss: 2.3023629188537598 Accuracy: 0.0703125
took 11.385411024093628 seconds

Loss: 1.7768718004226685 Accuracy: 0.390625
took 736.545031785965 seconds

Loss: 1.292565107345581 Accuracy: 0.5546875
took 738.4713110923767 seconds

Loss: 1.4291623830795288 Accuracy: 0.46875
took 745.3252530097961 seconds

Loss: 1.1174335479736328 Accuracy: 0.59375
took 736.8619091510773 seconds

Loss: 1.2858796119689941 Accuracy: 0.4921875
took 738.1686363220215 seconds

Loss: 1.1601579189300537 Accuracy: 0.5625
took 737.0851559638977 seconds

Loss: 1.1792998313903809 Accuracy: 0.5546875
took 740.1048610210419 seconds

Loss: 1.0095120668411255 Accuracy: 0.640625
took 736.0350155830383 seconds

Loss: 0.8251054883003235 Accuracy: 0.7109375
took 736.9550621509552 seconds

Loss: 0.7905967831611633 Accuracy: 0.6875
took 736.8096985816956 seconds

Loss: 0.7946950197219849 Accuracy: 0.7421875
took 735.6628801822662 seconds



In [0]:
test_acc_metric = tf.metrics.SparseCategoricalAccuracy()
for img_batch, lbl_batch in test_data:
    test_acc_metric(lbl_batch, model(img_batch))

print(test_acc_metric.result())

tf.Tensor(0.6782, shape=(), dtype=float32)


# With high level training loop

In [0]:
start = time.time()
model.compile(optimizer=opt, loss=loss_fn, metrics=[train_acc_metric])
model.fit(train_data, epochs=2)

stop = time.time()
print("took {} seconds\n".format(stop-start))


Epoch 1/2
1173/1173 [==============================] - 7042s 6s/step - loss: 1.3374 - sparse_categorical_accuracy: 0.5168
Epoch 2/2
1173/1173 [==============================] - 7036s 6s/step - loss: 0.8660 - sparse_categorical_accuracy: 0.6953
took 14094.14710855484 seconds



In [0]:
model.evaluate(test_data)

79/79 [==============================] - 102s 1s/step - loss: 0.9414 - sparse_categorical_accuracy: 0.6728


[0.9414253830909729, 0.6728000044822693]

# Experiments

With 3 - dense block each containing 4 conv layer of 32 3x3
                
    Training Accuracy = 69.53%
    Test Accuracy = 67.28

With 2- dense block each containing 4 conv layer of 32 3x3

    Training Accuracy = 68.34%
    Test Accuracy = 65.56%

With 2- dense block each containing 4 conv layer one with 32 3x3 and one with 64 3x3

    Training Accuracy = 74.21%
    Test Accuracy = 67.82%